# LLM Optimization Modelling Experiment

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [218]:
problem = '''A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared. 
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits. '''

## 2. Ask for parameters

In [234]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [235]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

* **X1:**  Number of 1.5L COLI units produced daily. 
* **X2:** Number of 1.5L PEPSA units produced daily.
* **S:** Hectoliters of syrup purchased and processed daily. 


# 2. Ask for objective

In [236]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [237]:
Markdown(response2.text)

Maximize  $49000X_1 - X_1^2 + 30X_2 - 2X_2^2 - 150S$ 


# 3. Ask for constraints

In [238]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [239]:
Markdown(response3.text)

Subject to:

*  $40X_1 + 20X_2  \leq 7100 \times 8$ (Hourly production capacity)
*  $X_2 \geq 0.5 X_1$ (Minimum PEPSA production)
*  $40X_1 + 20X_2 = 100S$ (Syrup conversion)
* $X_1 \geq 0$ (Non-negativity constraint)
* $X_2 \geq 0$ (Non-negativity constraint)
* $S \geq 0$ (Non-negativity constraint) 


## 3. Generate the pyomo code

In [240]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [241]:
#Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals) # Number of 1.5L COLI units produced daily
model.x2 = pyo.Var(domain=pyo.NonNegativeReals) # Number of 1.5L PEPSA units produced daily
model.S = pyo.Var(domain=pyo.NonNegativeReals)  # Hectoliters of syrup purchased and processed daily

# Define the objective function
def objective_rule(model):
  return 49000*model.x1 - model.x1**2 + 30*model.x2 - 2*model.x2**2 - 150*model.S
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
model.production_capacity = pyo.Constraint(expr= 40*model.x1 + 20*model.x2 <= 7100*8)
model.min_pepsa_production = pyo.Constraint(expr= model.x2 >= 0.5*model.x1)
model.syrup_conversion = pyo.Constraint(expr= 40*model.x1 + 20*model.x2 == 100*model.S)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # Select a solver (IPOPT is generally suitable for nonlinear problems)
results = solver.solve(model)

# Print the results
print("Solution Status:", results.solver.status)
print("Optimal Solution:")
print(f"X1 (COLI Production): {pyo.value(model.x1)}")
print(f"X2 (PEPSA Production): {pyo.value(model.x2)}")
print(f"S (Syrup): {pyo.value(model.S)}")
print(f"Maximum Profit: {pyo.value(model.profit)}") 
```


## 4. Input problem data and try running the generated code

In [242]:
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals) # Number of 1.5L COLI units produced daily
model.x2 = pyo.Var(domain=pyo.NonNegativeReals) # Number of 1.5L PEPSA units produced daily
model.S = pyo.Var(domain=pyo.NonNegativeReals)  # Hectoliters of syrup purchased and processed daily

# Define the objective function
def objective_rule(model):
  return 49000*model.x1 - model.x1**2 + 30*model.x2 - 2*model.x2**2 - 150*model.S
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
model.production_capacity = pyo.Constraint(expr= 40*model.x1 + 20*model.x2 <= 7100*8)
model.min_pepsa_production = pyo.Constraint(expr= model.x2 >= 0.5*model.x1)
model.syrup_conversion = pyo.Constraint(expr= 40*model.x1 + 20*model.x2 == 100*model.S)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # Select a solver (IPOPT is generally suitable for nonlinear problems)
results = solver.solve(model)

# Print the results
print("Solution Status:", results.solver.status)
print("Optimal Solution:")
print(f"X1 (COLI Production): {pyo.value(model.x1)}")
print(f"X2 (PEPSA Production): {pyo.value(model.x2)}")
print(f"S (Syrup): {pyo.value(model.S)}")
print(f"Maximum Profit: {pyo.value(model.profit)}") 

Solution Status: ok
Optimal Solution:
X1 (COLI Production): 1136.0000113639492
X2 (PEPSA Production): 568.0000056720345
S (Syrup): 568.0000056799865
Maximum Profit: 53660096.51744592


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [243]:
print(response.text)

## Variables:

* **X1:**  Number of 1.5L COLI units produced daily. 
* **X2:** Number of 1.5L PEPSA units produced daily.
* **S:** Hectoliters of syrup purchased and processed daily. 



In [244]:
print(response2.text)

Maximize  $49000X_1 - X_1^2 + 30X_2 - 2X_2^2 - 150S$ 



In [245]:
print(response3.text)

Subject to:

*  $40X_1 + 20X_2  \leq 7100 \times 8$ (Hourly production capacity)
*  $X_2 \geq 0.5 X_1$ (Minimum PEPSA production)
*  $40X_1 + 20X_2 = 100S$ (Syrup conversion)
* $X_1 \geq 0$ (Non-negativity constraint)
* $X_2 \geq 0$ (Non-negativity constraint)
* $S \geq 0$ (Non-negativity constraint) 



In [246]:
print(response4.text)

```python
import pyomo.environ as pyo

# Create a concrete model
model = pyo.ConcreteModel()

# Define variables
model.x1 = pyo.Var(domain=pyo.NonNegativeReals) # Number of 1.5L COLI units produced daily
model.x2 = pyo.Var(domain=pyo.NonNegativeReals) # Number of 1.5L PEPSA units produced daily
model.S = pyo.Var(domain=pyo.NonNegativeReals)  # Hectoliters of syrup purchased and processed daily

# Define the objective function
def objective_rule(model):
  return 49000*model.x1 - model.x1**2 + 30*model.x2 - 2*model.x2**2 - 150*model.S
model.profit = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Define constraints
model.production_capacity = pyo.Constraint(expr= 40*model.x1 + 20*model.x2 <= 7100*8)
model.min_pepsa_production = pyo.Constraint(expr= model.x2 >= 0.5*model.x1)
model.syrup_conversion = pyo.Constraint(expr= 40*model.x1 + 20*model.x2 == 100*model.S)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # Select a solver (IPOPT is generally suitable for nonlinear pr